# 時系列解析

本章では前章で確認した LSTM の実装をゴールとします。株価の実データを用い、時系列データの取り扱い、入力値・目標値の作成からモデルの構築などに取り組みます。

## 本章の構成

- 時系列データの取り扱い
- 株価の上昇・下落の 2 値分類の実装

※モデルの学習に GPU を使用する際にはランタイムの設定を変更しておきましょう。  

## 時系列データの取り扱い

時系列データの概要・基礎的な前処理について確認します。

### データの確認

今回は `yfinance` というパッケージを用いて株価の取得を行います。yfinance は pandas で取り扱うことのできるデータセットを外部から簡単に取得することが可能です。`yfinance` クラスを用い、Yahoo が提供する株価情報の取得を行います。  

In [ ]:
!pip install yfinance

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

In [ ]:
import tensorflow as tf
tf.__version__

株価の取得する初めと終わりの日付を定義します。  

In [ ]:
import datetime
date_st = datetime.datetime(2015, 1, 1)
date_fn = datetime.datetime(2021, 2, 1)

データセットの読み込みを行います。引数の `tickers` に株の銘柄を指定することにより、指定の銘柄を読み込む事が可能です。銘柄名に関しては「社名 銘柄」などのキーワードでご自身で検索し、確認して下さい。  

今回は Microsoft 社 (MSFT) の株価を使用します。

In [ ]:
df = yf.download('MSFT', date_st, date_fn)
df.head(3)

pandas の DataFrame 型でデータを読み込む事ができました。それぞれの列の説明は下記を確認して下さい。  

| 列名          | 日本語     | 説明                                                         |
| ------------- | ---------- | ------------------------------------------------------------ |
| **Date**      | 日付       | -                                                            |
| **High**      | 高値       | 1 日の最も高かった値段                                       |
| **Low**       | 安値       | 1 日の最も安かった値段                                       |
| **Open**      | 始値       | 1 日の始めについた値段                                       |
| **Close**     | 終値       | 1 日の終わりについた値段                                     |
| **Volume**    | 出来高     | 実際に売り買いが成立した株数                                 |
| **Adj Close** | 調整後終値 | [株式分割による調整後の値](https://www.yahoo-help.jp/app/answers/detail/p/546/a_id/45316) |

LSTM を用いて取り組む問題設定は前日の終値を用いて、次の日の終値が上がるか下がるかの 2 値の分類になります。終値を切り出し、ts (time series) という変数に格納します。また、  プロットし可視化します。  


In [ ]:
ts = df['Close']['MSFT']

In [ ]:
figsize = (14, 5) # プロットの大きさを定義
ts.plot(figsize=figsize)

In [ ]:
ts.shape

1258 日分のデータセットを用いて解析を行っていきます。

In [ ]:
ts.head()

### 移動平均

移動平均とは時系列データにおいて、ある一定区間ごとの平均値を区間をずらしながら求めたものです。移動平均を用いてグラフを作成すると、長期的な傾向を表す滑らかな曲線が得ることができます。（ノイズ除去したデータ）


`rolling(window)` メソッドでスライド窓 (window) での値を取ることができ、移動平均ではスライド窓の平均値を算出するので、 `mean()` メソッドで平均値をとります。

In [ ]:
moving_avgerage = ts.rolling(10).mean()
moving_avgerage.head(10)

上記の例ではスライド窓の幅を 10 と指定しているため、最初の 9 サンプルは NaN となっており、10 サンプル目から平均値が算出されている事が確認できます。

In [ ]:
moving_avgerage.plot(figsize=figsize)

プロット結果を確認すると先程プロットしたデータよりも滑らかになっていることが確認できます。トレンド性を確認する場合や、データセットからノイズを除去する場合などに用います。

### 差分

時系列データの代表的な前処理の 1 つ通して差分があります。この差分は時刻 $t$ 地点と時刻 $t−1$ 地点の差を使用します。差分を取ることで、上昇、下降などのトレンド性を取り除くことができます。時系列解析では、元のデータを原系列と呼び、差分をとったデータを差分系列と呼びます。  

pandas では `diff()` メソッドで差分を算出することができます。

In [ ]:
diff = ts.diff()
diff.head()

In [ ]:
diff.plot(figsize=figsize)

### 変化率

$t$ 地点と $t-1$ 地点でどれだけ値が変化したかを変化率と呼びます。
`pct_change()` メソッドを使い、算出することができます。$t$ 地点の値 $A$ と $t−1$ 地点の値を $B$ とすると変化率は $(A−B)/B$ と算出されます。もし周期性があれば、変化率が上下同じ幅で続いていくことが期待できます。

In [ ]:
pct_change = ts.pct_change()
pct_change.head()

In [ ]:
pct_change.plot(figsize=figsize)

### 入力・目標値の作成

時系列解析では、入力値、目標値作成が少しこれまでとは異なります。時系列解析では基本的に過去のデータを元に未来の値を予測するため、入力値には過去の値が、そして目標値には未来の値が入ります。  


#### LSTM 層の入力値

TensorFlow で LSTM を実装する際にはどの時刻（過去の期間）のデータを用いて、予測を行うのかの範囲の指定を行う必要があります。  

今回は練習として、$t$ 日の目 ~ $t+10$ 日の目の株価を入力にとり、$t + 11$ 日目の株価が上昇するか、下降するかを分類する問題設定に取り組みます。  

入力値を作成する際にはこの 10 日間を考慮してデータセットの作成を行う必要があります。TensorFlow の LSTM 層に対する入力値の形を確認し、作成すべきデータセットの形を理解します。  

![LSTM の入力](http://drive.google.com/uc?export=view&id=1qkcRUHXpA8H1X2WrukJHJi8yYWqD3l4e)

ニューラルネットワークのデータセットはサンプルと入力変数の行列の形式でした。入力値の形は入力変数の数の次元を持つベクトルになります。  
図の中では入力変数が a~h までの 8 個あるため、TensorFlow で入力値の形は `(8, )` で表現されます。  

次に LSTM の入力値の形を確認します。  
LSTM のデータセットはサンプル、入力変数に加え、**時刻 (time steps)** が加わり、テンソルになります。  
入力値の形は時刻と入力変数の数の行列になります。図内では上方向から確認できる行列になります。  

図の中の LSTM の入力値の形は `(5, 8)` と表現します。  

#### 時刻 (time steps) とは

ニューラルネットワークでは基本的に 1 サンプル 1 目標値のセットでした。LSTM でもこのセットの関係性は同じになります。しかし、LSTM では複数時刻の入力値と 1 つの目標値がセットとなってます。  

今回の株価の例を用いると、過去 10 日分の Microsoft 社の株価を用いて、翌日の株価を予測する場合、入力値の形は `(10, 1)` となります。このように複数時刻の入力値を使用して、予測を行うモデルを構築する場合、入力値と目標値の関係を考慮し、データセットを作成する必要があります。  

LSTM の時系列データの計算の流れを下記の図から確認しておきましょう。

![LSTM の計算の流れ](http://drive.google.com/uc?export=view&id=1n9XF-JPETetCKw8oEeNpUYshDx-9ozcT)

#### データセット作成

もう一度入力値と目標値の内容について確認します。  

- 入力変数 : $t$ 日の目 ~ $t+10$ 日の目の株価
- 目標値 : $t+11$ 日の目の株価が上がった場合 1 、下がったもしくは同じ場合は 0 の 2 種類のラベル  

入力値の作成方法から確認します。変数 `ts` から時刻 $t$ から時刻 $t+10$ をスライスし、1 つのサンプルとします。1 サンプルあたり、10 時刻、1 入力変数の形になります。


In [ ]:
# t ~ t+10 日目の株価の取得
time = 1 # 仮り置きの時刻
window = 10
ts[time : time + window]

仮り置きの時刻の値を変更して、時刻をずらしてデータを取得できる事を確認しましょう。  

続いて目標値の作成方法を確認します。目標値は $t+10$ と　$t+11$ の値を比較し、 0 もしくは 1 の値をとります。  

In [ ]:
# t+11 日目の株価の取得
time = 1
window = 10
ts.iloc[time + window]

In [ ]:
# t+10 日目の株価の取得
ts.iloc[time + window -1]

上記 2 つの値を比較することにより、目標値を作成します。

In [ ]:
ts.iloc[time + window] > ts.iloc[time + window -1]

上記のコードを for 文に当て込み、全てのデータセットに対して適用し、入力値、目標値の作成を行います。  
変数 `ts` は pandas の Series オブジェクトであるため、`values` 属性から NumPy の ndarray オブジェクトを取得します。

In [ ]:
window = 10
x, t = [], []
for time in range(len(ts) - window):
  x.append(ts[time : time + window].values)
  if ts.iloc[time + window] > ts.iloc[time + window -1]:
    t.append(1)
  else:
    t.append(0)

それぞれの値が正常に取得できているか確認します。下記のようにデータセットの作成を行った際には必ず重複などがデータセット内に含まれないこと確認します。予測対象のデータが入力値に含まれる事はよくある間違いなため、必ずダブルチェックを行いましょう。

In [ ]:
# t+10 日目の株価を取得
time = 5
x[time][-1]

In [ ]:
# t+11 日目の株価を取得
x[time+1][0]

In [ ]:
# 目標値の確認
t[time]

上記の例では株価が下がっているのに対し、目標値が 0 であることが確認でき、想定通りデータセットが作成されていることが確認できます。

今回のデータセットはサンプル毎に 1 時刻ずつずれています。（入力変数の数はどれも 1）  

- 1 サンプル目 : 時刻 1 ~ 10
- 2 サンプル目 : 時刻 2 ~ 11
- 3 サンプル目 : 時刻 3 ~ 12

それぞれのサンプルを時刻でスライスし、ズレを確認します。

In [ ]:
x[0][2], x[1][1], x[2][0]

リスト形式である `x` , `t` の値を NumPy の ndarray オブジェクトに変換を行います。

In [ ]:
x = np.array(x)
t = np.array(t)

In [ ]:
x.shape

LSTM 層の入力値としてこの形は望ましくありません。何故ならば、サンプル数、時刻数、入力変数の数を持ったテンソルの形である必要があるためです。そのため理想の形は `(1248, 10, 1)` という形になります。  

今回はまずはじめに全結合層のみのニューラルネットワークを用いて学習を行い、その後 LSTM 層を組み込みます。そのため、現段階ではこの行列の形を維持します。

最後にそれぞれのデータセットのデータ型の変換を行います。

In [ ]:
x = x.astype('float32')
t = t.astype('int32')

## 株価の上昇・下落の 2 値分類の実装

### データセットの分割

時系列データの学習・テスト用データセットへの分割を行う際に注意すべき点として、` train_test_split`の引数の `shuffle` を `False` と設定する必要があります。  
ランダムに分割を行ってしまうと、時系列データが持つ特有の前後関係が失われてしまいます。また、この際には `random_state` の引数の設定は必要ありません。

In [ ]:
from sklearn.model_selection import train_test_split

# 学習用データセットとテスト用データセットの分割
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.3, shuffle=False)

In [ ]:
# 形の確認
x_train.shape, x_test.shape, t_train.shape, t_test.shape

前後関係が保たれたままデータセットの分割が完了している事を確認しておきましょう。

In [ ]:
print('分割前のデータセット', x[0], x[1], x[2])
print('分割後のデータセット', x_train[0], x_train[1], x_train[2])

### モデルの定義（ニューラルネットワーク）

まずは全結合層のみのニューラルネットワークを使用して学習を行います。

In [ ]:
import os
import random

def reset_seed(seed=0):
    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
from tensorflow.keras import models, layers

# モデルの定義
reset_seed(0)

model = models.Sequential()

model.add(layers.Dense(5, input_shape=(10,), activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

### 目的関数と評価指標の選択

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

### モデルの学習


In [ ]:
history = model.fit(x_train, t_train,
          batch_size=100,
          epochs=20,
          validation_data=(x_test, t_test)
          )

### 予測精度の評価

In [ ]:
# 学習結果を取得
results = pd.DataFrame(history.history)
results.tail(3)

In [ ]:
# 損失を可視化
results[['loss', 'val_loss']].plot(title='loss')
plt.xlabel('epochs')

In [ ]:
# 正解率を可視化
results[['accuracy', 'val_accuracy']].plot(title='metric')
plt.xlabel('epochs')

損失・正解率ともに横ばいとなっており、学習が正常に完了していない事が確認できます。学習が正常に進んでいない原因を考えましょう。  

#### 問題点の考察

今回学習が正常に進んでいない原因・問題点を考えてみましょう。   
問題点としては下記の項目が挙げられます。  

- モデルの問題
  - データセットの時系列を考慮することができていない
- データセットの問題
  - 入力変数に使用するデータセットが適切でない

それぞれの項目がどういった内容か確認します。  


**モデルの問題**  

 まずは今回のニューラルネットワークのモデルがどのように計算されているか確認してみましょう。  

![NN](http://drive.google.com/uc?export=view&id=1g1yrBVh2TMePvgfG8nw_86mqF8_tQ5V7)

計算の流れを確認すると、それぞれの入力値が独立し予測値を計算している事がわかります。入力値が独立しているということは現在のニューラルネットワークのモデルでは時系列を考慮できていないということになります。

**データセットの問題**  

データセットに関しては単純に入力変数が足りていないことが考えられます。今回の株価を予測するためには、自社の過去の株価のみを入力とするのではなく、他社の株価やニュースの情報、原油価格など様々な項目があげる事ができるでしょう。

#### 対策の考察

上記の問題点への対策は下記になります。   

- LSTM (Long-Short Term Memory) モデルを用いる : 時系列を考慮したモデル
- 複数の入力変数を取る

それでは 1 つめの対策である LSTM のモデルの実装を行いましょう。

## LSTM（RNN）

時系列解析向けの Long-Short Term Memory （LSTM） を使用して結果の改善するか確認します。  


### LSTM 層の入力

先程確認した図をもう一度確認します。  

![LSTM の入力](http://drive.google.com/uc?export=view&id=1qkcRUHXpA8H1X2WrukJHJi8yYWqD3l4e)

現在のデータセットの形式は図の左側にあるニューラルネットワークのデータセットの形式になっています。ここから LSTM 層の入力値に対応させるためには入力変数の次元を追加する必要があります。  

現在のデータセットの入力変数の数は 1 つであるため、`(サンプル数, 時刻数, 1)` の形にデータセットを変換します。

In [ ]:
# Dense 層への入力値の形（サンプル数, 入力変数の数）
x.shape

In [ ]:
#  入力値の形の変換
x = x.reshape(len(x), 10, 1)

In [ ]:
# LSTM 層への入力値の形（サンプル数, 時刻の数, 入力変数の数）
x.shape

In [ ]:
# 学習用データセットとテスト用データセットの分割
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.3, shuffle=False)

### モデルの定義・学習・評価

データセットの準備が整いました。`layers` クラスの中から、`LSTM` 層を選択し、追加します。その後は同様の手順で学習から予測精度の確認までを行います。

In [ ]:
# LSTM 層を用いたモデルの構築
reset_seed(0)

model = models.Sequential()

model.add(layers.LSTM(5, input_shape=(10, 1), activation='tanh'))
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, t_train,
          batch_size=100,
          epochs=20,
          validation_data=(x_test, t_test)
          )

In [ ]:
# 学習結果を取得
results = pd.DataFrame(history.history)
results.head(3)

In [ ]:
# 損失を可視化
results[['loss', 'val_loss']].plot(title='loss')
plt.xlabel('epochs')

In [ ]:
# 正解率を可視化
results[['accuracy', 'val_accuracy']].plot(title='metric')
plt.xlabel('epochs')

LSTM 層を追加しても学習がうまくいきませんでした。考えられる原因としては下記が挙げられます。  
※あくまで考えられる原因の一部になります。  

- 時刻 (time steps) の幅が短すぎる
- 適切な入力変数を使用していない

上記以外にも様々な原因があることが考えられます。  


## 練習問題 本章のまとめ

本章で学んだ内容を復習しましょう。下記の内容を次のセルに記述し、実行結果を確認してください。（必要に応じてセルの追加を行ってください。）  
LSTM モデルのハイパーパラメータ調整やデータセットの作成に工夫を加え、再度 LSTM のモデルの作成を行って下さい。  

LSTM 層を増やす場合は return_sequences の引数を True に設定する必要があります。（各時刻ごとの隠れ状態ベクトルを次の層の LSTM に渡す必要があるため。）

*試行錯誤のポイント*  

- データセットの作成
  - 入力変数の数を増やす（他の企業の株価を使用する）
  - 時刻の幅を調整する
- モデルの調整
  - LSTM 層のノードの数
  - 全結合層の層・ノードの数
  - バッチノーマリゼーション層の追加
  - 最適化手法の変更
  - 学習係数の調整

*ヒント*  

- 複数社の株価の取得方法は下記のコード欄を確認して下さい。
- 入力値の形式は自由に変更して問題ありません。しかし、使用するのは列 `Adj Close` のみを使用して下さい。  
- 目標値は必ず Microsoft 社の株価が上がるか下がるかの 2 値であるようにして下さい。（複数社の値を入力変数に加えた場合も、目標値作成は Microsoft 社の株価の値を基準とします。）
- 株価の予測は過去の株価の影響のみから決定されるわけではありません。精度向上を目指すよりも、様々なアプローチを自分で考えることに重点を置きましょう。

In [ ]:
# リストに使用したい銘柄名を追加してください
symbols = ['MSFT', 'AAPL', 'GOOGL']

In [ ]:
df = yf.download(symbols, start=date_st, end=date_fn)
df['Close'].head()

### 入力値、目標値作成

複数社のデータを取扱う場合は先に NumPy の ndarray オブジェクトに変換しておくと処理が簡単になります。下記のコードの先程からの変更点は Microsoft 社のデータのみを抽出するために `ts[:, 0]` とスライスしている部分です。（全ての行の 1 列目を取得しています。）  

`window` 変数の値を変更し、学習結果にどのような変化があるか確認して下さい。  

In [ ]:
ts = df['Close'].values

In [ ]:
ts.shape

In [ ]:
ts[0:10]

In [ ]:
window = 10
x, t = [], []
for time in range(len(ts) - window):
  x.append(ts[time : time + window])
  if ts[:, 0][time + window] > ts[:, 0][time + window -1]:
    t.append(1)
  else:
    t.append(0)

In [ ]:
# データセットの形式の変換
x = np.array(x, 'f')
t = np.array(t, 'i')

In [ ]:
# サンプル数, 時刻数, 入力変数の数になっていることを確認
x.shape, t.shape

In [ ]:
# データセットの分割


In [ ]:
# LSTM 層を用いたモデルの構築


In [ ]:
# 目的関数と最適化手法の選択


In [ ]:
# モデルの学習


In [ ]:
# 学習結果を取得

In [ ]:
# 損失を可視化


In [ ]:
# 正解率を可視化


---
© 株式会社キカガク及び国立大学法人 豊橋技術科学大学